<!-- Projeto Conversa de PDFs -->

## <font color='blue'>IA Generativa e LLMs Para Processamento de Linguagem Natural</font>
## <font color='blue'>Projeto 1</font>
## <font color='blue'>Converse com Seus PDFs - Criando Assistente Pessoal de IA com LLM e VectorDB</font>

## Instalando e Carregando Pacotes

Para instalar todos os pacotes de uma vez, execute no terminal ou prompt de comando:

pip install -r requirements.txt

In [1]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -q -U watermark

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Bibliotecas utilizadas no projeto:
# - os: Para manipulação de caminhos e variáveis de ambiente.
# - openai: Para integração com a API da OpenAI.
# - langchain: Para construção de cadeias de processamento de linguagem natural.
# - chromadb: Para armazenamento e busca de vetores de embeddings.
# - langchain_community.document_loaders: Para carregar documentos em PDF.
# - langchain_community.embeddings: Para gerar embeddings de texto.
# - langchain_community.vectorstores: Para criar e gerenciar o armazenamento de vetores.
# - langchain_community.llms: Para integração com modelos de linguagem.
# - langchain.chains.question_answering: Para criar cadeias de perguntas e respostas.
# - warnings: Para ignorar avisos desnecessários.
# - dotenv: Para carregar variáveis de ambiente de um arquivo .env.

In [4]:
# Imports

import os
import openai
import langchain  
import chromadb 
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
import warnings
warnings.filterwarnings('ignore')



In [5]:
%reload_ext watermark
%watermark -a "Robson Brandão - CRAI"

Author: Robson Brandão - CRAI



## Extraindo Texto de Arquivos PDF

In [ ]:
# Função para ler os arquivos em pdf
def ins_read_pdf(directory_path):
    
    # Acessa a pasta com o arquivo pdf
    file_loader = PyPDFDirectoryLoader(directory_path)
    
    # Lê o documento da pasta
    documents = file_loader.load()
    
    return documents

In [ ]:
# Executa a função
ins_doc = ins_read_pdf('fo/')

In [ ]:
ins_doc

[Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-07-16T15:21:37-03:00', 'author': 'python-docx', 'moddate': '2025-07-16T15:21:37-03:00', 'source': 'fo\\Ciclos_1_6.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}, page_content='Série: Ciclos da Matéria – O Corpo Segundo Confúcio \nEsta obra apresenta uma abordagem integrada entre filosofia confucionista, estados físicos \nda matéria e a prática médico-cirúrgica moderna. Dividida em seis episódios, a série propõe \numa leitura vibracional e cíclica do corpo humano, do diagnóstico ao prognóstico, unindo \nciência, natureza e pensamento ancestral.'),
 Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-07-16T15:21:37-03:00', 'author': 'python-docx', 'moddate': '2025-07-16T15:21:37-03:00', 'source': 'fo\\Ciclos_1_6.pdf', 'total_pages': 15, 'page': 1, 'page_label': '2'}, page_content='Episódio 1 – A Filosofia do C

In [ ]:
# Número de páginas
len(ins_doc)

25

## Definindo a API do LLM

Crie sua API na OpenAI e coloque no arquivo .env na mesma pasta onde está este Jupyter Notebook.

https://platform.openai.com/

https://platform.openai.com/api-keys

https://platform.openai.com/docs/quickstart?context=python
<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
Veja mais detalhes no videobook do Capítulo 5 do curso.

In [10]:
# Carregando as variáveis de ambiente definidas no arquivo .env
from dotenv import load_dotenv
load_dotenv()



True

## Criando o Gerador de Embeddings

In [11]:
# Criando o gerador de embeddings
# embeddings são representações vetoriais de palavras ou frases que capturam seu significado semântico
# e são usados para medir a similaridade entre textos.
# Definindo a chave da API
gerador_embeddings = OpenAIEmbeddings(api_key = os.environ['OPENAI_API_KEY'])

O OpenAIEmbeddings é uma funcionalidade que permite obter representações vetoriais (embeddings) de texto, que são úteis para várias tarefas de processamento de linguagem natural, como comparação de semelhança de texto, agrupamento e classificação.

In [12]:
type(gerador_embeddings)

langchain_community.embeddings.openai.OpenAIEmbeddings

## Testando os Vetores de Embeddings

In [13]:
# Testando o gerador de embeddings
vetores = gerador_embeddings.embed_query('Qual habilidade mais importante na era da IA?')


In [14]:
type(vetores)

list

In [15]:
vetores[1:10]

[-0.00252506310742038,
 0.019988425955745487,
 -0.01234033456857347,
 -0.0014928358571401482,
 0.021698311766916074,
 -0.0206379172488283,
 0.0009568398347858864,
 -0.006074069234811833,
 -0.023673294810624815]

## Definindo o Vector Store

https://www.trychroma.com/

In [16]:
# Definindo o nome do índice
index_name = 'dsa-index'

In [ ]:
# Cria o vector store
index = Chroma.from_documents(ins_doc, gerador_embeddings, collection_name = index_name)

In [18]:
type(index)

langchain_community.vectorstores.chroma.Chroma

Veja mais detalhes sobre o Chroma no videobook do Capítulo 5 do curso.

In [ ]:
# Define a função de busca por similaridade
def ins_busca_similaridade(query, k = 2):
    
    # Utiliza o método 'similarity_search' do objeto 'index' para buscar os 'k' resultados 
    # mais semelhantes à 'query'
    matching_results = index.similarity_search(query, k)
    
    # Retorna os resultados correspondentes da busca de similaridade
    return matching_results

<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
## Criando a App do Assistente Pessoal com LangChain

In [20]:
# Cria instância do LLM
llm_dsa = OpenAI(openai_api_key = os.environ['OPENAI_API_KEY'], temperature = 0.3)

https://python.langchain.com/docs/modules/chains/document/stuff

https://js.langchain.com/docs/use_cases/question_answering/

In [21]:
# Cria a chain para perguntas e respostas em documentos
# stuff: carrega todo o documento na memória e faz a pergunta. stuff signifca qualuer coisa, ou seja,
# não se importa com o tamanho do documento, apenas carrega tudo na memória
# e faz a pergunta. Isso pode ser problemático para documentos muito grandes, pois pode causar
# estouro de memória.
# Isso é útil para documentos curtos ou quando a precisão não é crítica.
# map_reduce: divide o documento em partes menores, faz a pergunta em cada parte e depois combina as respostas
# Ref: https://python.langchain.com/docs/modules/chains/popular/qa
chain = load_qa_chain(llm_dsa, chain_type = 'stuff')

In [ ]:
# Define a função para obter resposta
def obter_resposta(query):

    # Chama a função 'ins_busca_similaridade' com a 'query' fornecida e armazena o resultado em 'doc_search'
    doc_search = ins_busca_similaridade(query)
        
    # Utiliza o objeto 'chain' para executar a função run e processar a 'query' e os documentos encontrados, 
    # armazenando a resposta em 'response'
    
    response = chain.run(input_documents = doc_search, question = query)
    
    # Retorna a resposta obtida do processamento anterior
    return response

## Executando o Assistente de IA e Conversando com PDFs

In [ ]:
# Pergunta para o arquivo pdf
ins_prompt1 = "como poderiamos pontuarm momentos de reflexão que avaliaem o aprendizado da técnica cirurgica"


In [ ]:
# Obtém a resposta
resposta1 = obter_resposta(ins_prompt1)

In [25]:
print(resposta1)


1. O professor enfatiza a importância do raciocínio clínico antes de realizar qualquer incisão.
2. O professor destaca a necessidade de uma técnica atraumática e respeito aos planos anatômicos.
3. O estudante aplica corretamente a anestesia infiltrativa, demonstrando compreensão da técnica.
4. O professor lista os princípios biológicos do reparo, reforçando a importância do conhecimento teórico antes da prática.
5. O estudante realiza a sutura, demonstrando habilidade adquirida durante o treinamento.
6. O professor avalia o desempenho do estudante e fornece feedback sobre possíveis melhorias na técnica cirúrgica.


In [ ]:
# Pergunta para o arquivo pdf
ins_prompt2 = "É possível você dar esses momentos de reflexão com os conceitos de patologia geral?"

In [ ]:
# Obtém a resposta
resposta2 = obter_resposta(ins_prompt2)

In [28]:
print(resposta2)

 Yes, the professor is discussing the principles of biological repair, which is a concept in general pathology. He is also discussing the difference between traumatic and atraumatic injuries, which is also a concept in general pathology.


In [ ]:
# Pergunta para o arquivo pdf
ins_prompt3 = "Apenas com esses elementos, é possível fazer uma análise crítica do aprendizado?"

In [ ]:
# Obtém a resposta
resposta3 = obter_resposta(ins_prompt3)

In [31]:
print(resposta3)

 Não, é necessário ter mais informações e contexto sobre a discussão e prática em questão para fazer uma análise crítica do aprendizado. Além disso, seria importante saber qual é o objetivo do aprendizado e se ele foi alcançado ou não.


In [32]:
%watermark -a "Robson Brandão - CRAI"

Author: Robson Brandão - CRAI



In [ ]:
# Pergunta para o arquivo pdf
ins_prompt4 = "É possivel fazer uma avaliação do aprendizado desse aluno com esses elementos ? Como ?"

In [ ]:
# Obtém a resposta
resposta4 = obter_resposta(ins_prompt3)

In [35]:
print(resposta4)

 Não, não é possível fazer uma análise crítica do aprendizado apenas com esses elementos. É necessário ter mais informações sobre o conteúdo abordado, a dinâmica da aula, a participação dos alunos e outros fatores para realizar uma análise crítica adequada.


In [ ]:
# Pergunta para o arquivo pdf
ins_prompt5 = "Qual o contexto e informações necessárias para fazer uma avaliação do aprendizado desse aluno com esses elementos ? Como ?"

In [ ]:
# Obtém a resposta
resposta5 = obter_resposta(ins_prompt3)

In [38]:
print(resposta5)

 No, com esses elementos não é possível fazer uma análise crítica do aprendizado. A discussão conceitual e a prática em tecido biológico são apenas uma parte do processo de aprendizado e não fornecem informações suficientes para uma análise crítica completa. Seria necessário ter acesso a mais informações sobre o contexto, como por exemplo, o objetivo do aprendizado, o nível de conhecimento prévio do estudante, a metodologia utilizada, entre outros. 


In [39]:
#%watermark -v -m

In [40]:
#%watermark --iversions

# Fim